In [1]:
!pip install datasets peft

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "google-t5/t5-large"

In [4]:
from datasets import load_dataset

dataset = load_dataset("lenatr99/Slovene_SuperGLUE_CB")

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
id2label = {0: "entailment", 1: "neutral", 2: "contradiction"}
label2id = {"entailment": 0, "neutral": 1, "contradiction": 2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
model.config.use_cache = False

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
CONTEXT_COL = "premise"
HYPOTHESIS_COL = "hypothesis"

def preprocess_function(examples):
    """
    The preprocessing function prepares examples for processing by the model.
    It concatenates premise and hypothesis for each example to form a single input string.
    """
    inputs = [f"{premise} {hypothesis}" for premise, hypothesis in zip(examples[CONTEXT_COL], examples[HYPOTHESIS_COL])]
    tokenized_examples = tokenizer(inputs, truncation=True)
    if "label" in examples:
        tokenized_examples["labels"] = [label2id[label] for label in examples["label"]]
    return tokenized_examples

In [8]:
tokenized_dataset = dataset.map(preprocess_function,
                                remove_columns=['idx', 'premise', 'hypothesis', 'label'], batched=True)

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 110
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 22
    })
})

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import numpy as np

from sklearn.metrics import accuracy_score, f1_score


def compute_metrics(pred):
    label_ids = pred.label_ids
    # Check if predictions is a tuple and take the first element if it is
    preds = pred.predictions[0] if isinstance(pred.predictions, tuple) else pred.predictions
    preds = preds.argmax(-1)
    f1 = f1_score(label_ids, preds, average="weighted")
    acc = accuracy_score(label_ids, preds)
    return {
        "accuracy": acc,
        "f1": f1,
    }


In [12]:
from transformers import set_seed

set_seed(42)

In [13]:
from peft import LoraConfig, get_peft_model, TaskType

lora_alpha = 32
lora_rank_dropout = 0.1
lora_module_dropout = 0.0
lora_r = 16

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=lora_r,
    lora_alpha=lora_alpha,
    bias="none",
    base_model_name_or_path=model_name
)

In [14]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 743,439,363 || trainable%: 0.6347


In [15]:
from transformers import TrainingArguments

new_model_name = "lora_fine_tuned_cb_t5"

training_args = TrainingArguments(
    output_dir=new_model_name,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    evaluation_strategy='steps',
    max_steps=400,
    use_cpu=False,
    load_best_model_at_end=True
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

max_steps is given, it will override any value given in num_train_epochs


In [17]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1
50,1.064300,1.233943,0.227273,0.150000
100,1.043800,1.223041,0.363636,0.227273
150,1.001000,1.211457,0.318182,0.153605
200,0.951600,1.198118,0.318182,0.153605
250,0.905800,1.193589,0.318182,0.153605
300,0.874600,1.193942,0.318182,0.153605
350,0.852000,1.197046,0.318182,0.153605
400,0.852400,1.197398,0.318182,0.153605


TrainOutput(global_step=400, training_loss=0.9431921482086182, metrics={'train_runtime': 224.4697, 'train_samples_per_second': 14.256, 'train_steps_per_second': 1.782, 'total_flos': 3454005272970768.0, 'train_loss': 0.9431921482086182, 'epoch': 28.571428571428573})

In [18]:
trainer.evaluate()

{'eval_loss': 1.1973979473114014,
 'eval_accuracy': 0.3181818181818182,
 'eval_f1': 0.1536050156739812,
 'eval_runtime': 0.9046,
 'eval_samples_per_second': 24.321,
 'eval_steps_per_second': 3.317,
 'epoch': 28.571428571428573}

In [19]:
trainer.push_to_hub(new_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


events.out.tfevents.1716199951.9f2d9443bbe2.1008.0:   0%|          | 0.00/8.58k [00:00<?, ?B/s]

events.out.tfevents.1716200068.9f2d9443bbe2.2072.0:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/18.9M [00:00<?, ?B/s]

events.out.tfevents.1716200293.9f2d9443bbe2.2072.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lenatr99/lora_fine_tuned_cb_t5/commit/423e1d909d1487a83f14af9a67b7bb4c551d3cee', commit_message='lora_fine_tuned_cb_t5', commit_description='', oid='423e1d909d1487a83f14af9a67b7bb4c551d3cee', pr_url=None, pr_revision=None, pr_num=None)

In [20]:
# Example
hypothesis ="Valence je pomagal"
premise = "Valence praznoglavi, Valence krepostni kreten. Zakaj si ga tip raje ni zataknil v ustrezen del lastne titanske anatomije? Je morda mislil, da mi pomaga?"

In [21]:
# We need to set the seed, otherwise some weights of the model are initialized differently every time, and consequently the result can be different each time as well
set_seed(42)
import torch

adapter_name = "lenatr99/" + new_model_name

tokenizer = AutoTokenizer.from_pretrained(adapter_name)
inputs = tokenizer(f"{premise} {hypothesis}", return_tensors="pt")
label = torch.tensor([1]).unsqueeze(0)  # Batch size 1

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [22]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
    trust_remote_code=True
)
outputs = model(**inputs, labels=label)
logits = outputs.logits

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google-t5/t5-large and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# Print prediction
logits.argmax().item()

0